# Vector ops scenario #1: YUV to RGB conversion

Example YUV to RGB conversion (simplified, not clamping)

**Note** Not verified for correct conversion, concept study only

Makes use of the `VectorSig` data type

In [1]:
# !pip install numpy

In [2]:
import sys
sys.path.insert(0, '../..')

In [3]:
from video.color import *
from video.videotypes import *

In [4]:
from myirl.library.bulksignals import bulkwrapper
from myirl import targets

In [5]:
CLAMP = False
LEVELSHIFT = False
BPP        = 8
FRACT_SIZE = 16
CALCSIZE   = FRACT_SIZE + BPP
SATURATION_VALUE_MAX = 127 # YUV maximum value (saturation)
SATURATION_VALUE_MIN = -128 # YUV minimum value (saturation)


# Signed matrix entries:
Y_FROM_RGB = vector_to_fp(FRACT_SIZE, 1, mat_jpeg_rgb2yuv[0])
U_FROM_RGB = vector_to_fp(FRACT_SIZE, 1, mat_jpeg_rgb2yuv[1])
V_FROM_RGB = vector_to_fp(FRACT_SIZE, 1, mat_jpeg_rgb2yuv[2])

def F(x, s = FRACT_SIZE):
    return intbv(x)[s:]

YUV_SLICE = slice(CALCSIZE-1, CALCSIZE-1 - BPP)

MATRIX = [
    [ F(Y_FROM_RGB[i]) for i in range(3) ],
    [ F(U_FROM_RGB[i]) for i in range(3) ],
    [ F(V_FROM_RGB[i]) for i in range(3) ]
]

from myirl.vector import VectorSig

I = lambda x: ( x[i]._val for i in range(3) )

# @bulkwrapper()
# class RGBParam:
#     def __init__(self):
#         self.y = VectorSig(3, MATRIX[0], initializer = I(MATRIX[0]))
#         self.u = VectorSig(3, MATRIX[1], initializer = I(MATRIX[1]))
#         self.v = VectorSig(3, MATRIX[1], initializer = I(MATRIX[2]))

In [6]:
MATRIX

[[intbv(9797), intbv(19234), intbv(3735)],
 [intbv(60007), intbv(54682), intbv(16384)],
 [intbv(16384), intbv(51817), intbv(62872)]]

In [7]:
from myirl.library.style_hdl import *
from myirl import simulation as sim
from myirl.test.common_test import gen_osc

@block
def video_rgb_yuv(clk : ClkSignal,
                  vin : VideoPort,
                  rgb : Signal,
                  param_matrix : list,
                  vout : VideoPort.Output,
                  yuv : Signal.Output,):
    """RGB to full range YUV422 converter"""

    v1 = vin.clone("vp_work")
    
    py, pu, pv = [
        VectorSig(3, F(0), initializer = I(param_matrix[i]), name = "p_coef%d" % i)  \
        for i in range(3)
    ]
    
    # Use initializers:
    py._init = True
    pu._init = True
    pv._init = True
    
    rgb_v = VectorSig(3, FractUnsigned(0, BPP), name = 'rgbv')
    
    a = VectorSig(3, FractSigned(0, CALCSIZE+2), name = "add_res")
    y = VectorSig(3, FractUnsigned(0, CALCSIZE), name = "ydata")
    u, v = [ VectorSig(3, FractSigned(0, CALCSIZE+1), name = n) for n in ['udata', 'vdata'] ]

    # Wire up input RGB video:
    wires = []
    for i in range(3):
        j = 3 - i
        wires.append(rgb_v[i].wireup(rgb[j*BPP:(j-1)*BPP]))


    # Predefine YUV slices
    yuv_slices = (a[i][YUV_SLICE] for i in range(3) )
         
    wires += [
        yuv.wireup(
            concat(*yuv_slices)
        )   
    ]
    
    @genprocess(clk, EDGE=clk.POS)
    def s1_mul():
        yield [        
            v1.set(vin), # Note (slight) inconsistency: '<=' not supported for bulks
            
            If (vin.dval == True) .Then(
                y <= py * rgb_v,
                u <= pu.signed() * rgb_v,
                v <= pv.signed() * rgb_v
            )
        ]

    @genprocess(clk, EDGE=clk.POS)
    def s1_acc():
        _y = y.sum()
        _u = u.sum()
        _v = v.sum()
        
        yield [        
            vout.set(v1),

            If (v1.dval == True) .Then(   
                a[0].set(_y.signed()),
                a[1].set(_u),
                a[2].set(_v)
            )
        ]

    return locals()


**Inconsistency Note**: Signal element assignment uses `.set`.

In [8]:
from myirl.targets import VHDL
from myirl.test.common_test import run_ghdl

@block
def testbench_rgb2yuv():
    clk = ClkSignal(name = "pclk")
    yuv = Signal(intbv(0)[3*BPP:])
    vint, vout = [VideoPort() for _ in range(2)]
    
    yuv = Signal(intbv(0)[3*BPP:], name = 'yuv_data')
    rgb = Signal(intbv(0)[3*BPP:], name = 'rgb_data')

    testbench_rgb2yuv.insert(vout)
    
    inst = video_rgb_yuv(clk = clk,
                        vin = vint,
                        rgb = rgb,
                        param_matrix = MATRIX,
                        vout = vout,
                        yuv = yuv
    )
    
    osc = gen_osc(clk, CYCLE = 5)
    
    @sim.generator
    def stimulus():       
        yield [
            sim.wait('1 ns'),
            rgb.set(0xff00ff),
            vint.dval.set(True), vint.fval.set(True), vint.lval.set(True),
            sim.wait(3 * [ clk.posedge, ] ), 
            sim.assert_(vout.dval == True, "Video not valid"),
        ]

        for _ in range(3):
            yield [
                sim.print_(yuv),
                sim.wait(clk.posedge), 
            ]

        
    return locals()

def test():
    tb = testbench_rgb2yuv()
    files = tb.elab(VHDL, elab_all = True)
    files.append('/tmp/record_defs.vhdl')
    video_rgb_yuv.ctx.create_library("record_defs")
    run_ghdl(files, tb, debug = True)

In [9]:
test()

VHDL target: REGISTERING <class 'myirl.library.bulksignals.VideoPort'>
Creating HDL style process 'video_rgb_yuv/s1_mul' with sensitivity (<clk>,)
Creating HDL style process 'video_rgb_yuv/s1_acc' with sensitivity (<clk>,)
 Insert unit video_rgb_yuv_s1dval_1_slval_1_sfval_1_s24__listdval_1_slval_1_sfval_1_s24 
Creating delay 'gen_osc/CLKGEN' : 5 ns
Creating sequential 'testbench_rgb2yuv/stimulus' 
 Insert unit testbench_rgb2yuv 


../../myirl/kernel/components.py:785: UserWarning: Already collected rgbv
  base.warnings.warn("Already collected %s" % n)
../../myirl/kernel/components.py:785: UserWarning: Already collected pclk
  base.warnings.warn("Already collected %s" % n)


 DEBUG: Writing 'video_rgb_yuv' to file /tmp/video_rgb_yuv.vhdl 
Finished _elab in 0.0917 secs


../../myirl/kernel/components.py:785: UserWarning: Already collected rgb_data
  base.warnings.warn("Already collected %s" % n)
../../myirl/kernel/components.py:1313: UserWarning: Type VideoPort already registered
  base.warnings.warn("Type %s already registered" % identifier)


 DEBUG: Writing 'testbench_rgb2yuv' to file /tmp/testbench_rgb2yuv.vhdl 
Finished _elab in 0.0319 secs
==== COSIM stdout ====

==== COSIM stderr ====

==== COSIM stdout ====
analyze /tmp/record_defs.vhdl
analyze ../../myirl/targets/../test/vhdl/txt_util.vhdl
analyze ../../myirl/targets/libmyirl.vhdl
analyze /tmp/video_rgb_yuv.vhdl
analyze /tmp/testbench_rgb2yuv.vhdl
elaborate testbench_rgb2yuv

==== COSIM stderr ====

==== COSIM stdout ====
0x69546A
0x69546A
0x69546A
./testbench_rgb2yuv:info: simulation stopped by --stop-time @1us

==== COSIM stderr ====



In [10]:
!cat -n /tmp/video_rgb_yuv.vhdl 

     1	-- File generated from /usr/local/lib/python3.8/runpy.py
     2	-- (c) 2016-2021 section5.ch
     3	-- Modifications may be lost
     4	
     5	library IEEE;
     6	use IEEE.std_logic_1164.all;
     7	use IEEE.numeric_std.all;
     8	
     9	library work;
    10	
    11	use work.record_defs.all;
    12	use work.txt_util.all;
    13	use work.myirl_conversion.all;
    14	
    15	entity video_rgb_yuv is
    16	    port (
    17	        clk : in std_ulogic;
    18	        vin : in t_VideoPort;
    19	        rgb : in unsigned(23 downto 0);
    20	        vout : out t_VideoPort;
    21	        yuv : out unsigned(23 downto 0)
    22	    );
    23	end entity video_rgb_yuv;
    24	
    25	architecture MyIRL of video_rgb_yuv is
    26	    -- Local type declarations
    27	    -- Signal declarations
    28	    signal vp_work : t_VideoPort;
    29	    type a_ydata is array (0 to 2) of unsigned(23 downto 0);
    30	    signal ydata : a_ydata    ;
    31	    type a_udata is array (0 to 2) of

In [11]:
!cat /tmp/record_defs.vhdl

-- File generated from /usr/local/lib/python3.8/runpy.py
-- (c) 2016-2021 section5.ch
-- Modifications may be lost

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

package record_defs is
    type t_VideoPort is record
        dval : std_ulogic;
        lval : std_ulogic;
        fval : std_ulogic;
    end record;

end package record_defs;

